# 分类算法

## 分类算法有哪些？

- 线性算法
  1. 逻辑回归
  2. 线性判别分析

- 非线性算法
  1. K近邻算法
  2. 贝叶斯分类器
  3. 分类与回归树
  4. 支持向量机

## 六种分类算法的比较

| 算法 | 特点 | 优点 | 缺点 | 最佳使用场景 |
|------|------|------|------|--------------|
| 逻辑回归 | 线性模型 | 易于实现和理解，计算效率高，输出结果具有概率解释 | 对非线性关系表现不佳，受到特征空间限制 | 二分类问题，特别是结果概率很重要的场景 |
| 线性判别分析(LDA) | 线性模型 | 适用于小样本数据集，可以用于降维 | 假设数据符合正态分布，同方差 | 数据线性可分，特征之间相关性较高的场景 |
| K近邻算法 | 非线性模型 | 实现简单，不需要训练阶段，适用于多分类问题 | 对大数据集和高维度数据计算成本高，对数据不均衡敏感 | 小数据集，实时决策，对数据分布无先验知识 |
| 贝叶斯分类器 | 概率模型 | 处理不确定性和不完整数据的能力强，适用于小数据集 | 基于先验知识，对输入数据敏感，计算复杂度高 | 数据维度高，先验知识丰富，数据不完整的场景 |
| 分类与回归树(CART) | 非线性模型 | 易于理解和解释，自动特征选择 | 容易过拟合，对数据变化敏感 | 决策需要解释性，特征选择重要，数据预处理要求不高 |
| 支持向量机(SVM) | 非线性模型 | 泛化能力强，有效处理高维数据，适用于复杂数据集 | 训练时间长，参数选择敏感 | 图像识别，文本分类，需要处理高维数据和非线性问题 |

有时，最好的方法是尝试几种不同的算法，并使用交叉验证等技术来比较它们的性能。

### 逻辑回归（Logistic Regression） 

#### 逻辑回归的基本概念

逻辑回归是一种用于**分类**的统计方法，尤其是用于**二分类**问题，比如“胖”或“瘦”。它的核心思想是找到一个**决策边界**（可以是一条直线、一个平面或一个超平面），将不同类别的数据分开。

#### 逻辑回归如何工作？

- **数据准备**：首先，我们有一些已知分类的**数据**（如身高、体重和对应的“胖”或“瘦”标签）。
- **模型训练**：逻辑回归试图找到一条**最佳的边界线**（或平面），这条边界能够最好地区分这些数据。
- **分类决策**：逻辑回归不是直接输出分类（如“胖”或“瘦”），而是输出一个介于0和1之间的**概率值**。例如，某个点的输出值是0.7，我们可以解释为70%的概率是“瘦”。

#### 何时使用逻辑回归？

- 当你的问题是二分类问题，比如是/否，真/假，胖/瘦等。
- 当你需要不仅知道分类结果，还想知道分类的概率时（比如，有多确信这个人是“瘦”的）。

#### 逻辑回归优点

1. **简单易懂**：逻辑回归是一种相对简单的模型，容易解释和理解。
2. **概率输出**：它提供了预测结果的概率评分，这对于需要权衡决策风险的场景非常有用。
3. **效率**：逻辑回归通常计算效率高，尤其适用于小到中等数据集。

#### 逻辑回归缺点

1. **处理非线性关系的能力有限**：逻辑回归假设数据是线性可分的，对于复杂或非线性可分的数据集表现不佳。
2. **对多类别分类支持不足**：虽然可以通过技术手段扩展到多类别分类，但逻辑回归本质上是**二分类**模型。

In [3]:
# 逻辑回归，在多分类算法评估模型中都有体现，这里与重复随机子抽样代码一样
from pandas import read_csv
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values # 读取数据
X = array[:, 0:8]
Y = array[:, 8]

n_splits = 10
test_size = 0.33
seed = 7
kfold = ShuffleSplit(n_splits = n_splits, test_size = test_size, random_state = seed)
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000) 
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 76.378% (2.316%)


### 线性判别分析（Linear Discriminant Analysis）

#### 线性判别分析（LDA）的基本概念

LDA的核心思想是找到一个最优的投影方向，使得在这个方向上，不同类别的数据分得越开越好，同时同一类别的数据尽可能紧密。

想象一下，你有一堆不同颜色的小球（每种颜色代表一个类别），你的任务是用一个灯光照射这些球，并调整灯光的角度，使得不同颜色的小球的影子尽可能分开，而相同颜色的球的影子尽可能靠近。在这个比喻中，灯光的角度就相当于LDA中的投影方向。

#### LDA如何工作？

- **数据准备**：同样地，我们从一些已知分类的数据开始，例如人的各种生理指标和对应的健康状况。
- **寻找最佳投影方向**：LDA试图找到一个方向，在这个方向上，**不同类别数据的中心（平均点）相距最远**，而**同类别数据的散布（方差）最小**。
- **数据投影**：一旦找到了这个方向，数据就被投影到这个新的维度上。在这个新的维度上，原先重叠和混杂的数据现在变得更加分明和可区分。

#### 何时使用LDA？

- 当你的数据集是**线性可分**的，特别是在你有较多特征（维度）需要区分两个或更多类别时。
- 在需要降维的场景中，特别是为了可视化或者提高算法效率。

#### LDA优点

1. **效率高**：LDA的计算效率相对较高，尤其适用于数据维度较高时。
2. **降维效果好**：LDA不仅是分类算法，也是一种有效的降维技术。

#### LDA缺点

1. **线性假设**：LDA假设数据是线性可分的，对于非线性数据效果不佳。
2. **正态分布假设**：LDA还假设每个类别的数据都近似正态分布，这在实际应用中可能不总是成立。
3. **同方差假设**：即假设所有类别的数据在每个特征上的方差都是相同的，这也可能在实际中不成立。

In [4]:
# 线性判别分析（LDA）
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score # 导入交叉验证的评估指标

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10 # K通常取3、5、10
seed = 7 # 随机种子，保证每次随机取样的结果一致
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = LinearDiscriminantAnalysis()
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 76.697% (4.797%)


### K近邻算法（K-Nearest Neighbors，简称KNN）

#### KNN的基本概念

KNN的核心思想非常简单：通过查看一个数据点的“邻居”来预测这个点的分类。就像在生活中，你往往可以通过观察一个人身边的朋友来了解这个人一样。

#### KNN如何工作？

- **没有显式学习过程**：与其他算法不同，KNN在训练阶段实际上并不计算什么，它只是存储数据。
- **分类决策**：当需要对一个新的样本进行分类时，KNN会在已存储的数据中找到与这个新样本最近的K个点。这个“最近”通常是用距离（如**欧几里得距离**）来衡量的。
- **投票机制**：一旦找到最近的K个点，算法就会根据这些邻居的类别进行投票，最多票的类别就被指定为新样本的类别。

#### 何时使用KNN？

- 当数据集不是太大时，因为KNN在大数据集上可能会很慢。
- 当我们没有足够的知识来选择更复杂的算法时，KNN是一个很好的起点。

#### KNN的优点

1. **简单直观**：KNN的原理非常简单，易于理解和实现。
2. **多功能**：可以用于分类和回归。
3. **不需要训练**：作为一种**懒惰算法**，KNN不需要构建和训练模型，只需存储数据。

#### KNN的缺点

1. **计算密集型**：对于大数据集，寻找最近邻点可能非常耗时。
2. **存储空间大**：因为算法需要存储所有训练数据。
3. **对不均衡数据敏感**：如果一个类别的样本比其他类别多很多，那么新样本更有可能被分类到这个多数类。

In [5]:
# k近邻算法
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score # 导入交叉验证的评估指标

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10 # K通常取3、5、10
seed = 7 # 随机种子，保证每次随机取样的结果一致
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = KNeighborsClassifier()
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 71.099% (5.079%)


### 贝叶斯分类器（Bayesian Classifier）

#### 贝叶斯分类器的基本概念

贝叶斯分类器基于贝叶斯定理，是一种利用概率统计知识进行分类的算法。它背后的核心思想是：我们可以利用已知的先验知识，结合新的观察数据，来更新我们对某个假设的信念。

#### 贝叶斯定理

贝叶斯定理是统计学中的一个重要定理，它告诉我们如何根据新的证据来更新概率。公式如下：

$$ P(A|B) = \frac{P(B|A) \times P(A)}{P(B)} $$

这里：
- $ P(A|B) $ 是后验概率：在 $ B $ 发生的条件下 $ A $ 发生的概率。
- $ P(B|A) $ 是似然概率：在 $ A $ 发生的条件下 $ B $ 发生的概率。
- $ P(A) $ 是 $ A $ 的先验概率：不考虑 $ B $ 的影响时 $ A $ 发生的概率。
- $ P(B) $ 是 $ B $ 的边际概率：在所有情况下 $ B $ 发生的总概率。

#### 贝叶斯分类器如何工作？

想象一下，你是一位侦探，正在试图根据一些线索（新的数据）来解决一个案件（进行分类）。你已经有一些基本的假设（先验概率），比如犯罪发生在夜间的可能性。现在，你获得了新的线索：比如，发现了一双泥泞的鞋子。这个新的证据会如何影响你对案件的理解（后验概率）？

#### 何时使用贝叶斯分类器？

- 当我们有关于类别的先验知识时。
- 在处理小数据集时，特别是当数据集的维度（特征数）较高时。

#### 贝叶斯分类器优点

1. **处理不确定性**：贝叶斯分类器非常适合处理具有**不确定性和不完整性**的数据。
2. **更新能力**：随着新数据的到来，模型可以被**更新**。
3. **对小数据集表现良好**：即使在数据较少的情况下，贝叶斯分类器也能给出合理的预测。

#### 贝叶斯分类器缺点

1. **先验知识的依赖**：其效果很大程度上取决于先验概率的准确性。
2. **对输入数据的敏感性**：贝叶斯分类器对输入数据的质量非常敏感。
3. **计算复杂度**：在特征数量增多时，模型的计算复杂度会增加。

In [6]:
# 朴素贝叶斯算法
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score # 导入交叉验证的评估指标

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10 # K通常取3、5、10
seed = 7 # 随机种子，保证每次随机取样的结果一致
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = GaussianNB()
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 75.914% (3.896%)


### 分类与回归树（CART）

#### 分类与回归树（CART）概念

CART是一种**决策树**学习方法，它可以用于分类问题（分类树）和回归问题（回归树）。决策树可以想象成一个问题树：每个内部节点代表一个问题或属性，每个分支代表问题的一个答案，而每个叶节点代表一个预测结果。

#### CART的特点

- **二叉树结构**：无论是分类还是回归，CART总是创建**二叉树**（每个节点最多有两个子节点）。
- **特征选择**：CART使用**基尼系数**（对于分类问题）和**平方误差**（对于回归问题）来选择最佳分割。

#### 基尼系数

在分类问题中，基尼系数是度量一个节点纯度的指标,***纯度高意味着节点中的大多数样本属于同一类别***。基尼系数代表了模型的不纯度，基尼系数越小，则不纯度越低（纯度越高），特征越好。

- **基尼系数的计算**：基尼系数表示在一个随机选中的样本被错误分类的概率。
- **选择最佳特征**：CART算法会选择使得分割后基尼系数下降最多的特征进行分割。

#### CART的工作原理

1. **特征选择**：首先，算法选择**最佳特征**和**分割点**来分割数据。
2. **创建二叉树**：根据选定的特征和分割点创建两个子节点。
3. **递归分割**：对每个子节点重复这个过程，直到满足停止条件（比如，节点中的所有样本都属于同一类别，或达到预设的深度限制）。
4. **剪枝**：为了避免过拟合，可能需要“剪掉”一些分支。

#### CART应用场景

- 分类问题：如判断邮件是否为垃圾邮件。
- 回归问题：如预测房屋价格。

#### CART优点

1. **直观易理解**：决策树模型易于理解和解释，**可以清楚地看到决策过程**。
2. **适用性广**：可以处理分类和回归问题。
3. **自动特征选择**：能够自动选出重要的特征，**对数据预处理要求不高**。

#### CART缺点

1. **过拟合倾向**：如果不剪枝，决策树可能过度复杂，对训练数据过拟合。
2. **稳定性差**：小的数据变动可能导致生成完全不同的树。

In [7]:
# 分类与回归树算法
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score # 导入交叉验证的评估指标

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10 # K通常取3、5、10
seed = 7 # 随机种子，保证每次随机取样的结果一致
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = DecisionTreeClassifier()
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 69.284% (5.102%)


### 支持向量机（SVM）

#### 支持向量机（SVM）的基本概念

SVM是一种**监督学习算法**，主要用于**二分类问题**。它的核心思想是找到一个最大边距超平面（决策边界），以最大化不同类别之间的间隔。

#### 线性SVM

在理想情况下，即数据线性可分的情况下，SVM寻找一个超平面，使得两个类别的数据被这个平面分开，并且到这个平面的最近点（支持向量）的距离最大。

想象一下，你在桌子上放了一些红球和蓝球，你需要拉一根绳子（超平面）来尽可能清晰地分开它们，而且你想让绳子离最近的红球和蓝球都有尽可能大的距离。

#### 非线性SVM

当数据不是线性可分的时，SVM通过一个被称为“核技巧”的过程将数据映射到**更高维度**的空间中，在这个新空间中，数据可能变得线性可分。这就像是你无法在桌面上直接分开红球和蓝球，但如果你能把它们投射到空中，就可能找到一个平面将它们分开。

### SVM的关键要素

1. **最大化边距**：SVM寻找的超平面是要最大化最近点（支持向量）之间的距离的。
2. **核技巧**：当数据不是线性可分时，SVM通过使用核函数将数据映射到更高维度的空间来寻找超平面。
3. **松弛变量**：在无法完美分割两类数据时引入，允许某些数据点违反最大边距原则。

### SVM优点

1. **泛化能力强**：SVM在**防止过拟合**方面表现出色，特别是在高维数据中。
2. **适用于复杂数据集**：通过核技巧，SVM能够处理**非线性可分的复杂数据集**。

### SVM缺点

1. **对大规模数据挑战**：在大规模数据集上，SVM的训练时间可能会很长。
2. **对参数选择敏感**：核函数的选择和参数设置（如C值）对最终结果有很大影响。

In [8]:
# 支持向量机算法
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.svm import SVC 
from sklearn.model_selection import cross_val_score # 导入交叉验证的评估指标 

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10 # K通常取3、5、10
seed = 7 # 随机种子，保证每次随机取样的结果一致
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = SVC()
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 76.046% (3.471%)
